## Load sampled images and download detailed metadata

In [1]:
import sys
sys.path.append("D:\\projects\\geo_image_experiments\\src")
import importlib
import mapillary_utils as mu
importlib.reload(mu)

<module 'mapillary_utils' from 'D:\\projects\\geo_image_experiments\\src\\mapillary_utils.py'>

In [15]:
import requests
from dotenv import load_dotenv
from os import getenv

from PIL import Image
from io import BytesIO
import pandas as pd
import numpy as np

import sqlite3
import json

import requests

pd.set_option('display.max_colwidth', None)

In [3]:
test_dir = "d://projects_working_directories//202407_compare_segmentations"
file = "segmentation_compare_2024-07-27.db"

images_dir = f"{test_dir}//images"

db_file = f"{test_dir}//{file}"

load_dotenv()
API_KEY = getenv("MAPILLARY_CLIENT_TOKEN")

In [4]:
# load sampled images 
conn = sqlite3.connect(f"{test_dir}//{file}")
df_sampled_images = pd.read_sql('select * from sampled_images', conn)
conn.close()

In [5]:
df_sampled_images

,image_lat,image_lon,residual,image_id,camera_type,is_pano,camera_focal_len,camera_k1,camera_k2,image_path,error,image_url
0,3.561633,98.637666,260.890344,137860785674785,perspective,0,0.270239,-0.077277115592856,0.0047516246006349,None,None,https://scontent.ffsd3-1.fna.fbcdn.net/m1/v/t6...
1,3.561741,98.637686,267.134725,114959664738493,perspective,0,0.398120,-0.11609096916052,0.0085084056129945,None,None,https://scontent.ffsd3-1.fna.fbcdn.net/m1/v/t6...
2,3.561876,98.637843,265.379210,999249780814409,perspective,0,0.270239,-0.077277115592856,0.0047516246006349,None,None,https://scontent.ffsd3-1.fna.fbcdn.net/m1/v/t6...
3,3.561831,98.637754,268.597539,5597563860322427,perspective,0,0.398120,-0.11609096916052,0.0085084056129945,None,None,https://scontent.ffsd3-1.fna.fbcdn.net/m1/v/t6...
4,3.561767,98.637705,267.550249,1294315551110501,perspective,0,0.398120,-0.11609096916052,0.0085084056129945,None,None,https://scontent.ffsd3-1.fna.fbcdn.net/m1/v/t6...
...,...,...,...,...,...,...,...,...,...,...,...,...
1225,3.629812,98.707824,173.430700,549610063194793,perspective,0,0.271258,-0.065300458067028,0.0045140519898224,None,None,https://scontent.ffsd3-1.fna.fbcdn.net/m1/v/t6...
1226,3.628367,98.710408,200.934439,656819662600366,perspective,0,0.309644,-0.094952229981501,0.0062149337092071,None,None,https://scontent.ffsd3-1.fna.fbcdn.net/m1/v/t6...
1227,3.628127,98.707901,253.149344,1812324055788633,perspective,0,0.580000,0,0,None,None,https://scontent.ffsd3-1.fna.fbcdn.net/m1/v/t6...
1228,3.628510,98.707933,220.093231,479574144154470,perspective,0,0.580000,0,0,None,None,https://scontent.ffsd3-1.fna.fbcdn.net/m1/v/t6...


In [6]:
#image_id = '856814222158020'

#image, metadata = mu.get_mapillary_image(image_id, API_KEY, image_size_indicator='thumb_2048_url', image_dir=images_dir)
#print("Metadata:", metadata.keys)

In [7]:
image_size_cycle = [
    'thumb_original_url','thumb_2048_url'
]

for image_id in df_sampled_images['image_id'].tolist():
    for image_size in image_size_cycle:
        image, metadata = mu.get_mapillary_image(image_id, API_KEY, image_size_indicator=image_size, image_dir=images_dir)
        try:
            df_temp = pd.DataFrame.from_dict([metadata])
            df_metadata = pd.concat([df_metadata,df_temp], ignore_index=True)
        except:
            df_metadata = pd.DataFrame.from_dict([metadata])

C:\Users\ericf\AppData\Local\Temp\ipykernel_11616\2790227054.py:10: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_metadata = pd.concat([df_metadata,df_temp], ignore_index=True)


In [8]:
def is_image_original_size(row):
    if (row.original_height == row.height) & (row.original_width == row.width):
        return 1
    else:
        return 0

df_metadata['is_image_original_size'] = df_metadata.apply(is_image_original_size, axis=1)

In [9]:
conn = sqlite3.connect(db_file)

df_metadata.to_sql('image_metadata', con=conn, if_exists='replace', index=False)



# Close the connection
conn.close()

In [17]:
df_metadata[['image_path_on_disk','camera_focal_len']][df_metadata['camera_focal_len'] > 0].sort_values(by='camera_focal_len', ascending=False)

,image_path_on_disk,camera_focal_len
1849,d://projects_working_directories//202407_compare_segmentations//images//mapillary_818312447084983_thumb_2048_url.jpg,2.931899
1848,d://projects_working_directories//202407_compare_segmentations//images//mapillary_818312447084983_thumb_original_url.jpg,2.931899
1847,d://projects_working_directories//202407_compare_segmentations//images//mapillary_400119589742621_thumb_2048_url.jpg,2.931899
1846,d://projects_working_directories//202407_compare_segmentations//images//mapillary_400119589742621_thumb_original_url.jpg,2.931899
914,d://projects_working_directories//202407_compare_segmentations//images//mapillary_1737449029972895_thumb_original_url.jpg,2.573973
...,...,...
109,d://projects_working_directories//202407_compare_segmentations//images//mapillary_201078782310182_thumb_2048_url.jpg,0.191538
108,d://projects_working_directories//202407_compare_segmentations//images//mapillary_201078782310182_thumb_original_url.jpg,0.191538
100,d://projects_working_directories//202407_compare_segmentations//images//mapillary_127192503472016_thumb_original_url.jpg,0.191538
392,d://projects_working_directories//202407_compare_segmentations//images//mapillary_5800898716598159_thumb_original_url.jpg,0.183734
